# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**

---

**Group:** P

**Names:**

* Matthias Leroy
* Pierre Fouche

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from scipy.sparse.linalg import svds

## Exercise 4.4: Latent semantic indexing

In [2]:
#We get from the disc the term-document matrix X
#as well as the dictionaries that link terms and document(courses) with their indices
with open("matrix.pickle", "rb") as f:
    X = pickle.load(f, encoding="utf-8")
with open("newCoursesDict.pickle", "rb") as f:
    newCoursesDict = pickle.load(f, encoding="utf-8")
with open("termsDict.pickle", "rb") as f:
    termsDict = pickle.load(f, encoding="utf-8")

# we apply SVD with K = 300 to our term-document matrix X.
U,S,Vt = svds(X,300)
print([e*e for e in S[-20:][::-1]])

(104639, 854)
[4.4181538238761719, 4.6419239235656162, 4.7537710437892269, 4.7827782397662917, 4.9981620096615922, 5.2173021453657586, 6.26553085166372, 6.448190922332012, 6.7546626229347169, 6.8834014191881305, 6.9742734820095853, 7.2165894288041423, 7.7336614220295745, 8.0869243549343786, 8.5190044656864714, 9.4664167579664049, 10.218845251935486, 19.871937944857287, 24.659548744702914, 32.007029864032283]
[  9.45167559e-03   3.39262459e-04  -1.55039323e-02  -2.79188107e-03
   1.84781529e-03  -4.92590093e-04   1.30908330e-03  -8.01908812e-04
   3.25252191e-03   9.51849384e-05]


1. Describe the rows and columns of U and V , and the values of S.
The columns of U describe a concept among all the terms of our matrix. Thus its rows define 

2. Print the top-20 eigenvalues of X.

## Exercise 4.5: Topic extraction

In [ ]:
topicTermDict = {}
indiceT = 1
for topicTerm in np.absolute(np.transpose(U))[-10:][::-1]:
    idxTerms = topicTerm.argsort()[-10:][::-1]
    tempList1 = []
    for idxT in idxTerms:
        term = termsDict[idxT]
        tempList1.append(term)
    topicTermDict[indiceT]=tempList1
    indiceT+=1
    
indiceD = 1
topicDocDict = {}
for topicDoc in np.absolute(Vt)[-10:][::-1]:
    idxDoc = topicDoc.argsort()[-10:][::-1]
    tempList2 = []
    for idxD in idxDoc:
        doc = newCoursesDict[idxD]['name']
        tempList2.append(doc)
    topicDocDict[indiceD]=tempList2
    indiceD+=1


for i in range(1,11):
    print('Terms for topic',i,'\n')
    for j in range(10):
        print(topicTermDict[i][j])
    print('\nDocument for topic',i,'\n')
    for k in range(10):
        print(topicDocDict[i][k])
    print('------------------------------------------------------------\n')


## Exercise 4.6: Document similarity search in concept-space

## Exercise 4.7: Document-document similarity